In [3]:
from pybis import Openbis
import json
import os
import ipywidgets as widgets
from IPython.display import display, Javascript, HTML
from datetime import datetime
from utils import read_json, close_notebook, AppWidgets

# TODO: Try to get a better approach
import sys
sys.path.append('/home/jovyan/aiida-openbis/Notebooks/importer')
import nanonis_importer

In [6]:
# Global variables
OPENBIS_SESSION = None
SESSION_DATA = None
RAW_MATERIALS_TYPES = ["CRYSTAL", "WAFER_SUBSTRATE", "2D_LAYERED_MATERIAL", "MOLECULE"]
PROCESS_SAMPLE_TYPES = ["SPUTTERING", "ANNEALING", "DEPOSITION"]

In [ ]:
# Widgets
app_widgets = AppWidgets("measurements", RAW_MATERIALS_TYPES, 
                         PROCESS_SAMPLE_TYPES)

In [8]:
# Connect with openBIS
app_widgets.connect_openbis()

# Measurement Uploader

## Select sample

In [ ]:
samples = app_widgets.openbis_session.get_objects(type = "SAMPLE")
instruments = app_widgets.openbis_session.get_objects(type = "INSTRUMENT")

samples_names_permids = [(sample.props['$name'], sample.permId) for sample in samples]
samples_dates = [datetime.fromisoformat(sample.registrationDate) for sample in samples]
samples_names_permids = [x for _,x in sorted(zip(samples_dates, samples_names_permids), reverse = True)]
samples_names_permids.insert(0, ('Select an input sample...', None))
app_widgets.samples_dropdown.options = samples_names_permids

instruments_names_permids = [(f"{instrument.props['$name']} ({instrument.attrs.permId})", instrument.permId) for instrument in instruments]
instruments_names_permids.insert(0, ('Select an instrument...', None))
app_widgets.instruments_dropdown.options = instruments_names_permids
app_widgets.instruments_dropdown.options = instruments_names_permids

display(app_widgets.sample_metadata_boxes, app_widgets.instruments_dropdown)
app_widgets.create_button.on_click(app_widgets.upload_measurements_to_openbis)
app_widgets.quit_button.on_click(close_notebook)
app_widgets.samples_dropdown.observe(app_widgets.load_sample_metadata, names = 'value')

## Select measurements folders

In [ ]:
display(app_widgets.folder_selector)
display(app_widgets.bottom_buttons_hbox)
display(app_widgets.increase_buttons_size)